In [54]:
import pandas as pd

In [55]:
data = pd.read_csv('covid_confirmed_usafacts.csv')

data = data.melt(id_vars=["countyFIPS","County Name", "State", "StateFIPS"], var_name="date", value_name="value")

data.head(4)


,countyFIPS,County Name,State,StateFIPS,date,value
0,0,Statewide Unallocated,AL,1,2020-01-22,0
1,1001,Autauga County,AL,1,2020-01-22,0
2,1003,Baldwin County,AL,1,2020-01-22,0
3,1005,Barbour County,AL,1,2020-01-22,0


In [56]:
data = data[['countyFIPS','value','date', 'County Name', 'State']]

data['date'] = pd.to_datetime(data['date'])

data = data.rename(columns={"countyFIPS": "fips", "value": "act_cases", "County Name": "county", "State": "state"})

confirmed_cases = data

data.head(10)


,fips,act_cases,date,county,state
0,0,0,2020-01-22,Statewide Unallocated,AL
1,1001,0,2020-01-22,Autauga County,AL
2,1003,0,2020-01-22,Baldwin County,AL
3,1005,0,2020-01-22,Barbour County,AL
4,1007,0,2020-01-22,Bibb County,AL
5,1009,0,2020-01-22,Blount County,AL
6,1011,0,2020-01-22,Bullock County,AL
7,1013,0,2020-01-22,Butler County,AL
8,1015,0,2020-01-22,Calhoun County,AL
9,1017,0,2020-01-22,Chambers County,AL


In [57]:
item = []
for i in range(6):
    item.append('fcast_data' + str(i+1) + '.csv')    

print(item)

data = pd.concat(map(pd.read_csv, item), ignore_index=True)

['fcast_data1.csv', 'fcast_data2.csv', 'fcast_data3.csv', 'fcast_data4.csv', 'fcast_data5.csv', 'fcast_data6.csv']


C:\Users\ChrisEvans\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [58]:
#data = data[data.target == "1 wk ahead inc case"]
#data = data[data.location_name != data.State]
data['date'] = pd.to_datetime(data['target_end_date'])

data = data.rename(columns={'point': "for_cases"})

data = data[['model','date','fips','for_cases','target']]

forcasted_cases = data

forcasted_cases.head(10)


,model,date,fips,for_cases,target
0,BPagano,2020-10-24,1,8924,1 wk ahead inc case
1,BPagano,2020-10-24,2,3554,1 wk ahead inc case
2,BPagano,2020-10-24,4,9092,1 wk ahead inc case
3,BPagano,2020-10-24,5,6166,1 wk ahead inc case
4,BPagano,2020-10-24,6,24521,1 wk ahead inc case
5,BPagano,2020-10-24,8,7660,1 wk ahead inc case
6,BPagano,2020-10-24,9,2029,1 wk ahead inc case
7,BPagano,2020-10-24,10,745,1 wk ahead inc case
8,BPagano,2020-10-24,11,421,1 wk ahead inc case
9,BPagano,2020-10-24,12,22717,1 wk ahead inc case


In [59]:
forcasted_cases['fips'] = forcasted_cases['fips'].astype(str)
forcasted_cases['date'] = forcasted_cases['date'].astype(str)

confirmed_cases['fips'] = confirmed_cases['fips'].astype(str)
confirmed_cases['date'] = confirmed_cases['date'].astype(str)

all_cases = pd.merge(forcasted_cases, confirmed_cases, on=['date','fips'], how='inner')

all_cases.head(10)

,model,date,fips,for_cases,target,act_cases,county,state
0,Columbia,2020-08-08,2013,0,1 wk ahead inc case,2,Aleutians East Borough,AK
1,Ensemble,2020-08-08,2013,0,1 wk ahead inc case,2,Aleutians East Borough,AK
2,Columbia,2020-08-08,2016,0,1 wk ahead inc case,4,Aleutians West Census Area,AK
3,Ensemble,2020-08-08,2016,2,1 wk ahead inc case,4,Aleutians West Census Area,AK
4,Columbia,2020-08-08,2020,745,1 wk ahead inc case,2175,Municipality of Anchorage,AK
5,Ensemble,2020-08-08,2020,589,1 wk ahead inc case,2175,Municipality of Anchorage,AK
6,LANL,2020-08-08,2020,589,1 wk ahead inc case,2175,Municipality of Anchorage,AK
7,Columbia,2020-08-08,2050,0,1 wk ahead inc case,22,Bethel Census Area,AK
8,Ensemble,2020-08-08,2050,2,1 wk ahead inc case,22,Bethel Census Area,AK
9,LANL,2020-08-08,2050,2,1 wk ahead inc case,22,Bethel Census Area,AK


In [60]:
all_cases.to_csv('all_cases.csv')